# .NET Aspire 🤩

In [ ]:
on dotnet/aspnetcore milestone 9; check out theme item
    - https://github.com/dotnet/aspnetcore/issues/53178

- https://github.com/dotnet/aspnetcore

<img src=images/aspire-overview.png width=800>

- https://learn.microsoft.com/en-us/dotnet/aspire/fundamentals/app-host-overview

- https://learn.microsoft.com/en-us/dotnet/aspire/fundamentals/setup-tooling
- https://learn.microsoft.com/en-us/dotnet/aspire/get-started/build-your-first-aspire-app
- https://learn.microsoft.com/en-us/dotnet/core/extensions/generic-host

# .NET 💖 AI

- AI is "cool" with C#/.NET/Azure
- .NET 💖 AI
    - Microsoft.Data.Analysis and ML.NET: Bert (TorchSharp), Onnx
    - Model Builder
    - Community Data Science / AI / ML Libraries
    - C# and F#

### Microsoft.Data.Analysis

In [ ]:
#r "nuget:Microsoft.Data.Analysis"
using Microsoft.Data.Analysis;

In [ ]:
using Microsoft.Data.Analysis;

var series = new PrimitiveDataFrameColumn<int>("MySeries");

series.Append(1);
series.Append(2);
series.Append(3);

Console.WriteLine(series.Length);
Console.WriteLine(series[0]);       // it works like array + something more

series.CumulativeProduct() // pandas has cumprod; multiply value with previous value; 1 as is; 2 will become 2 x 1; 3 will become 3 x 2

In [ ]:
// its .NET flavored; implements IEnumerable and hence you can use LINQ
series.OrderByDescending(s => s).Take(1).FirstOrDefault();
// linq and relational queries are good for row level computation
// but its become hard to do reduce like operations where at row operation has some sense of positional context

// in nutshell; you are getting best of both worlds

What's wrong here?
- You have to learn new OOPish API
- C/C++/Java/C# are last choices for Data Scientists; they like non ceremonial languages (scripty / untypish)
    - C# is becoming succint but its still a ceremonial language;
- Python Ecosystem is stronger and its OK
    - *Python and Node workloads in Visual Studio; but who uses it?* 😇

## F#

- .NET Interactive and Polyglot Notebooks
- Charting Libraries: XPlot, Plotly.NET
- Libraries
    - Deedle; https://fslab.org/Deedle/tutorial.html
    - TorchSharp originated in Xamarin and F#
    - https://diffsharp.github.io
    - https://numerics.mathdotnet.com
    - SharpCV (Computer Vision)
    - and others
- http://fslab.org

### Math Things in F#

In [1]:
#r "nuget: Plotly.NET.Interactive"
#r "nuget: MathNet.Numerics.FSharp, 5.0.0"

open Plotly.NET
open MathNet.Numerics.LinearAlgebra

In [2]:
let xData = [ 0. .. 10. ]
let yData = [ 0. .. 10. ]

Chart.Point(xData, yData)
|> Chart.withTitle "Hello world!"
|> Chart.withXAxisStyle ("xAxis")
|> Chart.withYAxisStyle ("yAxis")

In [3]:
Chart.Line(xy = [ for x in 1.0 .. 100.0 -> (x, x ** 2.0) ])

In [4]:
let m = matrix [[ 1.0; 4.0; 7.0 ]
                [ 2.0; 5.0; 8.0 ]
                [ 3.0; 6.0; 9.0 ]]

let v = vector [ 10.0; 20.0; 30.0 ]

let v' = m * v
let m' = m + 2.0*m

### TorchSharp

In [5]:
#r "nuget: TorchSharp-cpu"

open TorchSharp
open type torch
open type TorchSharp.TensorExtensionMethods
open type TorchSharp.torch.distributions
open type TorchSharp.torch.nn


In [6]:
// Bernoulli distrubtion; false/true or 0/1 (coin toss)
let bern = Bernoulli(torch.tensor(0.5f))
bern.sample(3L,4L)

In [7]:
let linear_layer = nn.Linear(inputSize = 3, outputSize = 1)
linear_layer.bias

<img src=images/torch-linear.png>

#### F# and Python

In [ ]:
// Python example from https://pytorch.org/tutorials/beginner/basics/buildmodel_tutorial.html
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

- https://www.ml-science.com/rectifier-activation-function

<img src=images/torch-activation.png>

In [ ]:
type NeuralNetwork() =
    inherit nn.Module("NeuralNetwork")

    let flatten = nn.Flatten()
    let linear_relu_stack = nn.Sequential (
        nn.Linear(28L *28L, 512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.ReLU(),
        nn.Linear(512, 10)
    )

    let forward(x) =
        let x = flatten.forward(x)
        let logits = linear_relu_stack.forward(x)
        logits

In [1]:
// Restart Kernel
#r "nuget: TorchSharp-cpu"
using TorchSharp;
using static TorchSharp.torch.nn;

In [2]:
var lin1 = Linear(1000, 100);
var lin2 = Linear(100, 10);
var seq = Sequential(("lin1", lin1), ("relu1", ReLU()), ("drop1", Dropout(0.1)), ("lin2", lin2));

var x = torch.randn(64, 1000);
var y = torch.randn(64, 10);

var optimizer = torch.optim.Adam(seq.parameters());

for (int i = 0; i < 10; i++) {
    using var eval = seq.forward(x);
    using var output = functional.mse_loss(eval, y, Reduction.Sum);

    optimizer.zero_grad();

    output.backward();

    optimizer.step();
}

- https://github.com/dotnet/TorchSharpExamples
    - https://github.com/dotnet/TorchSharpExamples/tree/main/tutorials

### DiffSharp

In [3]:
#r "nuget: DiffSharp-cpu"
open DiffSharp

In [4]:
let t1 = dsharp.tensor [ 0.0 ..0.2.. 1.0 ] // Gives [0., 0.2, 0.4, 0.6, 0.8, 1.]
let t2 = dsharp.tensor [ 1, 2, 3, 4, 5, 6 ]

t1 + t2

In [ ]:
let t3 = dsharp.tensor [[[[0.0 .. 10.0]]]]
let t4 = dsharp.tensor [[[[0.0 ..0.1.. 1.0]]]]

// think of convolution as a filter for smoothing out signal or filtering something; dimensionality reduction
// t3 as input; t4 as output; if we want convolution between these two; we should get another vector
t3.conv2d(t4)
// https://www.ml-science.com/convolution

In [ ]:
let f (x: Tensor) = x.exp().sum() // f is objective function

// we are interested how it changes with respect to its input tensor
// how much each component of tensor is contributing to the change in the value of 'f'
// this info is used in ML optimization techniques / gradient descent

dsharp.grad f (dsharp.tensor([1.8, 2.5])) // Gradient of the vector-to-scaler
// https://www.ml-science.com/gradients

<img src=images/gradient-descent.png>

- https://en.wikipedia.org/wiki/Gradient_descent
- https://diffsharp.github.io

### F# as a Better Python ?

- F# as a Better Python - https://www.youtube.com/watch?v=_QnbV6CAWXc

<img src=images/ai-net-2.jpg width=800>

- Python Tools in Visual Studio
- https://developer.microsoft.com/en-us/python
    - Python and VS Code
    - Python in SQL Server
    - Python in Excel

__Resources__
- https://www.cambridge.org/core/books/functional-programming-using-f/BAB494BBE8AEBB13486F9A8362273D41
- https://fsharpforfunandprofit.com
- https://exercism.org/tracks/fsharp
- https://www.fssnip.net
- https://dotnet.microsoft.com/en-us/learn/fsharp

## Semantic Kernel

- https://learn.microsoft.com/en-us/semantic-kernel/overview
- https://github.com/microsoft/semantic-kernel
    - .NET, Python and Java

<img src=images/sk.png>

In [ ]:
#r "nuget:Microsoft.SemanticKernel"

In [ ]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;

var builder = Kernel.CreateBuilder();

builder.AddOpenAIChatCompletion(
         "gpt-3.5-turbo",                                       // OpenAI Model name
         "");                                                   // OpenAI Key
var kernel = builder.Build();

var prompt = @"{{$input}}

One line TLDR with the fewest words.";

var summarize = kernel.CreateFunctionFromPrompt(prompt, executionSettings: new OpenAIPromptExecutionSettings { MaxTokens = 100 });

string text1 = @"
1st Law of Thermodynamics - Energy cannot be created or destroyed.
2nd Law of Thermodynamics - For a spontaneous process, the entropy of the universe increases.
3rd Law of Thermodynamics - A perfect crystal at zero Kelvin has zero entropy.";

string text2 = @"
1. An object at rest remains at rest, and an object in motion remains in motion at constant speed and in a straight line unless acted on by an unbalanced force.
2. The acceleration of an object depends on the mass of the object and the amount of force applied.
3. Whenever one object exerts a force on another object, the second object exerts an equal and opposite on the first.";

Console.WriteLine(await kernel.InvokeAsync(summarize, new() { ["input"] = text1 }));

Console.WriteLine(await kernel.InvokeAsync(summarize, new() { ["input"] = text2 }));

// Output:
//   Energy conserved, entropy increases, zero entropy at 0K.
//   Objects move in response to forces.

<img src=images/gpt-scenario.png width=800>

- Travel Plan Example: https://chat.openai.com/share/a21b3b0e-d293-4728-bca1-4e12eb647dc8
- https://www.linkedin.com/pulse/power-semantic-kernel-azure-openai-sap-data-chanjin-park-y8u3c

__Further Resources__
- https://github.com/microsoft/semantic-kernel/blob/main/dotnet/README.md
- 👉 https://github.com/microsoft/semantic-kernel/tree/main/dotnet/samples
    - https://github.com/microsoft/semantic-kernel/tree/main/dotnet/samples/HomeAutomation
    - https://github.com/microsoft/semantic-kernel/tree/main/dotnet/samples/HuggingFaceImageTextExample

<img src=images/rag.png width=800>

<img src=images/lang-chain.jpg width=800>

### LangChain vs Semantic Kernel

| Type                   | LangChain's Agents | Semantic Kernel's Planner |
| ---------------------- | ------------------ | ------------------------- |
| Conversational         | ✅                 | ❌                        |
| Plan and execute       | ✅                 | ✅ (SequentialPlanner)    |
| ReAct                  | ✅                 | ✅ (StepwisePlanner)      |
| Tree of Thoughts (ToT) | ✅                 | ❌                        |

<img src=images/sk-overview.png>

- 👉 Demo https://www.youtube.com/watch?v=TNqAJUgm-sU
- https://devblogs.microsoft.com/semantic-kernel/building-ai-applications-with-microsoft-semantic-kernel-and-mongodb-atlas-vector-search
- https://github.com/microsoft/SemanticKernelCookBook

# Azure 💖 AI

- https://azure.microsoft.com/en-us/products/ai-services

## Azure AI Services (Cognitive Service) 🪂

<img src=images/azure-ai.png width=900>

<img src=images/chat-gpt-business.png width=800>

- Language
    - Intent
- Translation
- Content Safety
    - Detecting Offensive content
- Speech
    - Transcribe, translate and generate audio
- Vision
    - OCR
- Document Intelligence
    - Upload document and get meta data

Pricing
- https://azure.microsoft.com/en-us/pricing/details/cognitive-services

<img src=images/train-model.png width=800>

- Azure OpenAI Service
    - Why not use OpenAI when Azure OpenAI is slightly expensive?
        - As per internet; Azure OpenAI is slightly faster (time it takes for prompts to process)
        - Azure OpenAI latency is slightly less
    - Mostly inconclusive
- Azure AI Search
    - Full Text + Vector Search / Hybrid
    - https://learn.microsoft.com/en-us/azure/search/retrieval-augmented-generation-overview
    - Using this; you can do something like JFK Example; https://www.youtube.com/watch?v=6kw8SHwxp9c
    - Pricing Model is similar to Cosmos; storage + search units
    - Think of it like Elasticsearch /w AI
        - ML features in new Elasticsearch / Elastic Cloud


## Analytics 🤑

### Overview

<img src=images/big-data-logical.svg>

- https://learn.microsoft.com/en-us/azure/architecture/guide/architecture-styles/big-data

<img src=images/azure-big-data.png width=900>

<img src=images/azure-big-data-2.png width=900>

### Products

- Azure Synapse Analytics
- Databricks
- HD Insights
- Azure Data Factory
- Azure Stream Analytics
- Azure Machine Learning

and others; https://azure.microsoft.com/en-us/products/category/analytics

### Databricks

<img src=images/azure-databricks.png>

### Azure Synapse

<img src=images/azure-synapse.webp width=900>

<img src=images/synapse-1.avif width=900><br>
<img src=images/synapse-2.avif>

### Knowledge Mining

- Analytics + AI
- https://azure.microsoft.com/en-us/solutions/knowledge-mining

<img src=images/azure-knowledge-minning.png width=900>

## Azure Machine Learning 🤑

<img src=images/azure-ml.png width=800>

<img src=images/azure-mlops.png width=800>

- https://ml.azure.com (Firefox / KN Profile)

__Resources__
- https://learn.microsoft.com/en-us/azure/machine-learning/concept-model-catalog
    - https://learn.microsoft.com/en-us/azure/machine-learning/how-to-use-automl-onnx-model-dotnet

## Azure AI Studio 😎🤑

<img src=images/azure-ai-studio.webp width=800>

<img src=images/copilot-stack.png width=800>

- https://ai.azure.com (Firefox / KN Profile)

__Resources__
- Azure AI Studio Demo - Microsoft Ignite 2023: https://www.youtube.com/watch?v=-6nN4M9juIo
- https://github.com/Azure-Samples/aistudio-python-langchain-sample?tab=readme-ov-file

## Microsoft Fabric 🥰 🤑

<img src=images/ms-fabric.jpg width=800>

<img src=images/ms-fabric-users.jpg width=800>

<img src=images/ms-fabric-expanded.jpg width=800>

<img src=images/data-processing.jpg width=800>

# Poor Man's Vector Search

__Inception__
- https://sl.bing.net/ZU3u43AUP6

__Elaboration__
- https://neptune.ai/blog/tokenization-in-nlp
- https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2

__Construction__
- https://github.com/curiosity-ai/hnsw-sharp
- https://github.com/curiosity-ai/MiniLM
    - https://github.com/NMZivkovic/BertTokenizers
- Demo

__Transition__
- https://sl.bing.net/iu6M8Mbz4o0

# Succeeding with AI

### Approach

<img src=images/ai-taxonomy.jpg width=800><br>

<img src=images/ai-taxonomy-2.jpg width=800>

- Late Payment Example: https://chat.openai.com/share/a34b846e-8f3f-4fd4-b27a-30e25793ab25

### Team

<img src=images/data-team.jpg>

# Resources 📚

## AI Architecture 🌱

<img src=images/azure-arch-ai.svg><br>
https://learn.microsoft.com/en-us/azure/architecture/databases/idea/intelligent-apps-using-azure-database-for-postgresql

<img src=images/azure-arch-ecomm.png width=900><br>
https://learn.microsoft.com/en-us/azure/architecture/example-scenario/apps/ecommerce-search


<img src=images/azure-arch-bot.svg><br>
https://learn.microsoft.com/en-us/azure/architecture/solution-ideas/articles/custom-business-processes

- lot more @ - https://learn.microsoft.com/en-us/azure/architecture/browse

__Learning__
- https://learn.microsoft.com/en-us/training/career-paths/data-analyst
- https://learn.microsoft.com/en-us/training/paths/data-analytics-microsoft
- https://learn.microsoft.com/en-us/training/paths/understand-data-concepts
    - https://learn.microsoft.com/en-us/training/modules/get-started-data-analytics

__Github__
- https://github.com/SciSharp/Numpy.NET
- https://github.com/dotnet/csharp-notebooks/tree/main/machine-learning
- https://github.com/mdfarragher/DSC
- https://github.com/microsoft/Data-Science-For-Beginners

__Youtube__
- https://www.youtube.com/@dotnet
    - Generative AI with .NET for Beginners Playlist (12 Videos): https://www.youtube.com/watch?v=vISLS8aY0RU&list=PLdo4fOcmZ0oW_k4_eDTPWDLUVWz7A9y0M
    - ML.NET Model Builder: Machine learning with .NET: https://www.youtube.com/watch?v=cUqNzZwzUV0